In [1]:
!pip install -U --quiet mlfoundry
!pip install -U --quiet servicefoundry

     |████████████████████████████████| 126 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 3.4 MB 39.6 MB/s 
     |████████████████████████████████| 281 kB 68.4 MB/s 
     |████████████████████████████████| 2.1 MB 36.3 MB/s 
     |████████████████████████████████| 181 kB 57.6 MB/s 
     |████████████████████████████████| 132 kB 41.3 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 9.1 MB 33.3 MB/s 
     |████████████████████████████████| 140 kB 37.9 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 1.6 MB 54.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 127 kB 53.5 MB/s 
     |████████████████████████████████| 410 kB 54.6 MB/s 
     |████████████████████████████████| 183 kB 69.8 MB/s 
     |█████████████████

In [8]:
!sfy login

You are logged in as 'srihari' with email 'srihari@truefoundry.com'



## Building a webapp using our models

In [18]:
%%writefile webapp.py

import streamlit as st
import mlfoundry
import pandas as pd

from scipy.sparse import coo_matrix

@st.cache(allow_output_mutation=True)
def load_models_and_dfs():
  client = mlfoundry.get_client()
  # REPLACE THIS WITH YOUR RUN FQN
  run = client.get_run('truefoundry/srihari/movie-recommendation/burgundy-emu')

  # REPLACE THIS WITH YOUR MODEL FQN
  model = client.get_model("model:truefoundry/srihari/movie-recommendation/reco-implicit:1").load()


  movies_local_path = run.download_artifact('movies_metadata.csv')
  ratings_local_path = run.download_artifact('ratings_small.csv')

  movie_meta_df = pd.read_csv(movies_local_path)
  ratings_df = pd.read_csv(ratings_local_path)

  movie_meta_df = movie_meta_df[movie_meta_df['id'].isin(ratings_df['movieId'].astype('string'))]
  
  ratings_df['movieId_cat'] = ratings_df['movieId'].astype("category")
  ratings_df['userId_cat'] = ratings_df['userId'].astype("category")

  ratings = ratings_df['rating']
  rows = ratings_df['userId_cat'].cat.codes
  cols = ratings_df['movieId_cat'].cat.codes

  sparse_matrix = coo_matrix((ratings, (rows, cols)))
  return model, sparse_matrix, movie_meta_df, ratings_df

model, sparse_matrix, movie_meta_df, ratings_df = load_models_and_dfs()

@st.cache(allow_output_mutation=True)
def get_movie_id_from_cat_code(cat_code):
  return ratings_df['movieId_cat'].cat.categories[cat_code]

@st.cache(allow_output_mutation=True)
def get_user_id_from_cat_code(cat_code):
  return ratings_df['userId_cat'].cat.categories[cat_code]

@st.cache(allow_output_mutation=True)
def get_cat_code_from_user_id(user_id):
  return ratings_df['userId_cat'].cat.categories.get_loc(user_id)

@st.cache(allow_output_mutation=True)
def get_cat_code_from_movie_id(movie_id):
  return ratings_df['movieId_cat'].cat.categories.get_loc(movie_id)

@st.cache(allow_output_mutation=True)
def search_movie(name):
  return (movie_meta_df.loc[movie_meta_df['original_title'].str.contains(name, case=False)][['original_title', 'id']]).to_dict('records')


def get_similar_movies(movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = int(search_result[0]['id'])
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = get_cat_code_from_movie_id(movie_id)
  return [get_movie_id_from_cat_code(cat_code) for cat_code in model.similar_items(movie_cat_code)[0]]

def get_recommendation_for_user(user_id):
  user_cat_code = get_cat_code_from_user_id(user_id)
  return [get_movie_id_from_cat_code(cat_code) for cat_code in model.recommend(user_cat_code, sparse_matrix.tocsr().getrow(user_cat_code))[0]]
  
def get_movie_names_for_movie_ids(movie_ids):
    return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in movie_ids])].original_title)

tab1, tab2 = st.tabs(["Similar Movies", "Recommend for User"])

with tab1:
  movie_name = st.selectbox('Movie title', list(movie_meta_df['original_title'].head(50)))
  st.write('Similar Movies')
  for movie_id in get_movie_names_for_movie_ids(get_similar_movies(movie_name)):
      st.markdown("- " + movie_id)


with tab2:
  user_id = st.selectbox('Enter User Id', list(ratings_df['userId'].unique()))
  st.write('Recommendations for user')
  for movie_id in get_movie_names_for_movie_ids(get_recommendation_for_user(user_id)):
      st.markdown("- " + movie_id)

Overwriting webapp.py


In [12]:
%%writefile requirements.txt
streamlit
implicit
scipy
pandas
mlfoundry

Overwriting requirements.txt


In [13]:
%%writefile .sfyignore
sample_data/
.config/

Overwriting .sfyignore


In [19]:
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

logging.basicConfig(level=logging.INFO)

image = Build(
    build_spec=PythonBuild(
        command="streamlit run webapp.py --server.port 8080",
        requirements_path="requirements.txt",
    ),
)

# REPLACE THIS WITH YOUR SECRET
env = {
    "TFY_API_KEY": "tfy-secret://srihari:tfy-secrets:TFY_API_KEY"
}

service = Service(
    name="streamlit-app",
    image=image,
    ports=[{"port": 8080}],
    env=env,
    resources=Resources(cpu_request='0.5', cpu_limit='0.5', memory_limit='1000', memory_request='1000' )
)

deployment = service.deploy(workspace_fqn="v1:tfy-cluster-euwe1:ut-demo-ws")

In [24]:
%%writefile main.py

from fastapi import FastAPI

import mlfoundry
import pandas as pd

from scipy.sparse import coo_matrix

def load_models_and_dfs():
  client = mlfoundry.get_client()
  # REPLACE THIS WITH YOUR RUN FQN
  run = client.get_run('truefoundry/srihari/movie-recommendation/burgundy-emu')

  # REPLACE THIS WITH YOUR MODEL FQN
  model = client.get_model("model:truefoundry/srihari/movie-recommendation/reco-implicit:1").load()


  movies_local_path = run.download_artifact('movies_metadata.csv')
  ratings_local_path = run.download_artifact('ratings_small.csv')

  movie_meta_df = pd.read_csv(movies_local_path)
  ratings_df = pd.read_csv(ratings_local_path)

  movie_meta_df = movie_meta_df[movie_meta_df['id'].isin(ratings_df['movieId'].astype('string'))]
  
  ratings_df['movieId_cat'] = ratings_df['movieId'].astype("category")
  ratings_df['userId_cat'] = ratings_df['userId'].astype("category")

  ratings = ratings_df['rating']
  rows = ratings_df['userId_cat'].cat.codes
  cols = ratings_df['movieId_cat'].cat.codes

  sparse_matrix = coo_matrix((ratings, (rows, cols)))
  return model, sparse_matrix, movie_meta_df, ratings_df

model, sparse_matrix, movie_meta_df, ratings_df = load_models_and_dfs()

def get_movie_id_from_cat_code(cat_code):
  return ratings_df['movieId_cat'].cat.categories[cat_code]

def get_user_id_from_cat_code(cat_code):
  return ratings_df['userId_cat'].cat.categories[cat_code]

def get_cat_code_from_user_id(user_id):
  return ratings_df['userId_cat'].cat.categories.get_loc(user_id)

def get_cat_code_from_movie_id(movie_id):
  return ratings_df['movieId_cat'].cat.categories.get_loc(movie_id)

def search_movie(name):
  return (movie_meta_df.loc[movie_meta_df['original_title'].str.contains(name, case=False)][['original_title', 'id']]).to_dict('records')


def get_similar_movies(movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = int(search_result[0]['id'])
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = get_cat_code_from_movie_id(movie_id)
  return [get_movie_id_from_cat_code(cat_code) for cat_code in model.similar_items(movie_cat_code)[0]]

def get_recommendation_for_user(user_id):
  user_cat_code = get_cat_code_from_user_id(user_id)
  return [get_movie_id_from_cat_code(cat_code) for cat_code in model.recommend(user_cat_code, sparse_matrix.tocsr().getrow(user_cat_code))[0]]
  
def get_movie_names_for_movie_ids(movie_ids):
    return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in movie_ids])].original_title)

app = FastAPI()

@app.get("/recommend")
def predict(user_id: int):
  return get_movie_names_for_movie_ids(get_recommendation_for_user(user_id))

Overwriting main.py


In [22]:
%%writefile requirements.txt
fastapi
uvicorn
implicit
scipy
pandas
mlfoundry

Overwriting requirements.txt


In [25]:
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

logging.basicConfig(level=logging.INFO)

image = Build(
    build_spec=PythonBuild(
        command="uvicorn main:app --port 8080 --host 0.0.0.0",
        requirements_path="requirements.txt",
    ),
)

# REPLACE THIS WITH YOUR SECRET
env = {
    "TFY_API_KEY": "tfy-secret://srihari:tfy-secrets:TFY_API_KEY"
}

service = Service(
    name="fastapi-svc",
    image=image,
    ports=[{"port": 8080}],
    env=env,
    resources=Resources(cpu_request='0.5', cpu_limit='0.5', memory_limit='1000', memory_request='1000' )
)

deployment = service.deploy(workspace_fqn="v1:tfy-cluster-euwe1:ut-demo-ws")